In [ ]:
# -*- coding: utf-8 -*-
# 카탈로그 정제 · 패턴 탐지 프로젝트 (가공식품 소매가격 정보 기반)

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest

# ------------------------------------------------
# 1. 데이터 로드
# ------------------------------------------------
# encoding은 환경에 따라 cp949 / utf-8 중 하나로 설정
df = pd.read_csv("c:/Users/soule/Downloads/한국농수산식품유통공사_가공식품_소매가격_정보_20250826.csv")

# 날짜 형변환
df["조사일"] = pd.to_datetime(df["조사일"])

# 기본 정보 확인
print(df.head())
print(df.info())

# ------------------------------------------------
# 2. 기본 특성 파악
# ------------------------------------------------
print("전체 행 수:", len(df))
print("품종(상품명) 개수:", df["품종"].nunique())
print("품목 목록:", df["품목"].unique())

# 요약 통계 (품목·품종 기준)
summary = (
    df.groupby(["품목", "품종"])[["평균가", "최저가", "최고가"]]
      .agg(["mean", "min", "max"])
      .round(1)
)
print(summary)

# ------------------------------------------------
# 3. 상품명 텍스트 패턴 분석 (TF-IDF + KMeans)
# ------------------------------------------------
# 3-1. TF-IDF 벡터화
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df["품종"])

# 3-2. KMeans 군집화
k = 5
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(X)

print("\n[클러스터별 대표 품종 예시]")
for c in range(k):
    examples = df[df["cluster"] == c]["품종"].unique()[:5]
    print(f"\n=== Cluster {c} ===")
    for e in examples:
        print(" -", e)

# ------------------------------------------------
# 4. 텍스트 기반 이상치 탐지 (Isolation Forest)
# ------------------------------------------------
# 간단한 텍스트 피처 설계
df["name_len"] = df["품종"].str.len()
df["digit_cnt"] = df["품종"].str.count(r"\d")
df["unit_flag"] = df["품종"].str.contains("g|kg|ml|mL|L|ℓ|리터|봉|개", regex=True).astype(int)

feature_cols = ["name_len", "digit_cnt", "unit_flag"]
X_feat = df[feature_cols]

iso = IsolationForest(contamination=0.03, random_state=42)
df["name_anomaly"] = iso.fit_predict(X_feat)  # -1: 이상치, 1: 정상

name_anoms = df[df["name_anomaly"] == -1]
print("\n[상품명 기반 이상치 탐지 결과]")
print("탐지된 이상 상품 행 수:", len(name_anoms))
if len(name_anoms) > 0:
    print(name_anoms[["조사일", "품목", "품종", "name_len", "digit_cnt", "unit_flag"]].head())

# ------------------------------------------------
# 5. 가격 시계열 이상치 탐지 (Z-score)
# ------------------------------------------------
price_group = (
    df.groupby(["조사일", "품목", "품종"])["평균가"]
      .mean()
      .reset_index()
)

def add_zscore(group):
    mu = group["평균가"].mean()
    sigma = group["평균가"].std()
    if sigma == 0:
        group["z"] = 0
    else:
        group["z"] = (group["평균가"] - mu) / sigma
    return group

price_z = price_group.groupby(["품목", "품종"], group_keys=False).apply(add_zscore)

# |z| > 3 인 날을 이상치로 판단
price_anoms = price_z[price_z["z"].abs() > 3].copy()
print("\n[가격 이상치 탐지 결과 (|z| > 3)]")
print("가격 이상치 행 수:", len(price_anoms))
print(price_anoms.sort_values(["품목", "품종", "조사일"]).head(20))

# ------------------------------------------------
# 6. (옵션) 결과 저장
# ------------------------------------------------
df.to_csv("catalog_pattern_detection_result.csv", index=False, encoding="utf-8-sig")
price_anoms.to_csv("price_anomaly_result.csv", index=False, encoding="utf-8-sig")
print("\n결과 파일 저장 완료:")
print(" - catalog_pattern_detection_result.csv")
print(" - price_anomaly_result.csv")
